In [ ]:
# Set up 

In [2]:
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import sqlalchemy as db
from sqlalchemy import create_engine
import datetime
import sqlite3
import pandas as pd
from sqlalchemy.sql.schema import Column


In [3]:
start = pd.to_datetime('2018-01-01')
end = pd.to_datetime('today')

FXAIX_stock = web.DataReader('FXAIX', 'yahoo', start, end)
FXAIX_stock.head()

SP500_stock = web.DataReader('^GSPC', 'yahoo', start, end)
SP500_stock.head()

Russell_2000_stock = web.DataReader('^RUT', 'yahoo', start, end)
Russell_2000_stock.head()


stocks = pd.concat([SP500_stock['Open'], Russell_2000_stock['Open'], FXAIX_stock['Open']], axis = 1)

stocks.columns = ['SP500_stock','Russell_2000_stock', 'FXAIX_stock']

stocks.reset_index(level=0, inplace=True)
stocks

engine = db.create_engine('sqlite:///stocks.db')
connection = engine.connect()
metadata = db.MetaData()


stocks_table = db.Table('index_stocks_table', metadata, 
    db.Column('Date',db.Date, nullable=True, index=False),
    db.Column('SP500_stock',db.Integer, nullable=True),
    db.Column('Russell_2000_stock',db.Integer, nullable=True),
    db.Column('FXAIX_stock', db.Numeric, nullable=True)
)

metadata.create_all(engine) 

stocks.to_sql('index_stocks_table', con=engine, if_exists='replace', index=False)

sql = "SELECT * FROM index_stocks_table LIMIT 10"
cnxn = connection
df = pd.read_sql(sql, cnxn)
print(df.head(10))


                         Date  SP500_stock  Russell_2000_stock  FXAIX_stock
0  2018-01-02 00:00:00.000000  2683.729980         1536.119995    94.230003
1  2018-01-03 00:00:00.000000  2697.850098         1550.280029    94.830002
2  2018-01-04 00:00:00.000000  2719.310059         1552.979980    95.230003
3  2018-01-05 00:00:00.000000  2731.330078         1555.869995    95.900002
4  2018-01-08 00:00:00.000000  2742.669922         1559.800049    96.059998
5  2018-01-09 00:00:00.000000  2751.149902         1562.219971    96.220001
6  2018-01-10 00:00:00.000000  2745.550049         1559.010010    96.110001
7  2018-01-11 00:00:00.000000  2752.969971         1560.219971    96.790001
8  2018-01-12 00:00:00.000000  2770.179932         1587.119995    97.449997
9  2018-01-16 00:00:00.000000  2798.959961         1592.910034    97.110001


```

#!/bin/bash
source ~/anaconda3/etc/profile.d/conda.sh
conda activate base
python stocks_fiax.py
datasette publish heroku -n stocks-snp-500 stocks.db --install=datasette-vega

```

In [ ]:
#snp_list_tables.columns = ['Symbol','Security', 'SEC filings', 'GICS Sector', 'GICS Sub-Industry', ' Headquarters Location', 'Date first added', 'CIK', 'Founded']
#snp_list_tables.head(10)
#
#
#snp_list_table = db.Table('snp_list_tables', metadata, 
#    db.Column('Symbol',db.String, nullable=True, index=False),
#    db.Column('Security',db.String, nullable=True),
#    db.Column('SEC filings',db.String, nullable=True),
#    db.Column('GICS Sub-Industry',db.String, nullable=True),
#    db.Column('Headquarters Location',db.String, nullable=True)
#    , db.Column('Date first added', db.Date, nullable=True)
#    , db.Column('CIK', db.String, nullable=True)
#    , db.Column('Founded', db.Date, nullable=True)
#)
#
#snp_list_table.to_sql('snp_list', con=engine, if_exists='replace', index=False)
#
#sql = "SELECT * FROM snp_list LIMIT 10"
#cnxn = connection
#df = pd.read_sql(sql, cnxn)
#print(df.head(30))
